In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import timedelta
from datetime import datetime
from datetime import datetime
import networkx as nx

In [3]:
trips_may = pd.read_csv('yellow_tripdata_2016-05.csv')
trips_june = pd.read_csv('yellow_tripdata_2016-06.csv')
trips = pd.concat([trips_may, trips_june])

In [4]:
# Filter trips: only consider trips with 0 < distance < 100
trips = trips.loc[(trips['trip_distance'] > 0.0) & (trips['trip_distance'] < 100.0), :]

In [5]:
# Function to calculate distance between drop off points of 2 trips
def distanceTwoDPO(trip1, trip2):
    pku_long_rad = trip1.dropoff_longitude*np.pi/180
    pku_lat_rad = trip1.dropoff_latitude*np.pi/180
    dpo_long_rad = trip2.dropoff_longitude*np.pi/180
    dpo_lat_rad = trip2.dropoff_latitude*np.pi/180
    drad = 2*np.arcsin(np.sqrt(np.sin((pku_lat_rad-dpo_lat_rad)/2)*np.sin((pku_lat_rad-dpo_lat_rad)/2)+np.cos(pku_lat_rad)*np.cos(dpo_lat_rad)*np.sin((pku_long_rad-dpo_long_rad)/2)*np.sin((pku_long_rad-dpo_long_rad)/2)))
    dmile = drad*180*60*1.15/np.pi
    return dmile

In [6]:
# Function to calculate Euclidean distance from pickup point to dropoff point of 1 trip
def distance(df):
    pku_long_rad = df['pickup_longitude']*np.pi/180
    pku_lat_rad = df['pickup_latitude']*np.pi/180
    dpo_long_rad = df['dropoff_longitude']*np.pi/180
    dpo_lat_rad = df['dropoff_latitude']*np.pi/180
    drad = 2*np.arcsin(np.sqrt(np.sin((pku_lat_rad-dpo_lat_rad)/2)*np.sin((pku_lat_rad-dpo_lat_rad)/2)+np.cos(pku_lat_rad)*np.cos(dpo_lat_rad)*np.sin((pku_long_rad-dpo_long_rad)/2)*np.sin((pku_long_rad-dpo_long_rad)/2)))
    dmile = drad*180*60*1.15/np.pi
    return df.assign(distance=dmile)

In [7]:
# Create new column in DataFrame named distance
trips = trips.pipe(distance)

In [8]:
# Only consider trips with Euclidean distance in range (0, 100)
trips = trips.loc[(trips['distance'] > 0.0) & (trips['distance'] < 100.0), :]

In [9]:
# Calculate distance ratio = trip distance / Euclidean distance
trips['distance_ratio'] = trips['trip_distance']/trips['distance']

In [10]:
# Only consider trips with distance ratio in range (0, 100)
trips = trips.loc[(trips['distance_ratio'] > 0.0) & (trips['distance_ratio'] < 100.0), :]

In [11]:
# Convert pickup and dropoff datetime to datetime type
trips['pku_time'] = pd.to_datetime(trips['tpep_pickup_datetime'], infer_datetime_format=True)
trips['dpo_time'] = pd.to_datetime(trips['tpep_dropoff_datetime'], infer_datetime_format=True)

In [12]:
trips = trips[['trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'distance', 'distance_ratio', 'pku_time', 'dpo_time']]

In [13]:
trips = trips.loc[(trips['dpo_time']-trips['pku_time'])/timedelta(seconds=1) > 0.0, :]

In [14]:
# Calculate average speed of a trip
trips['avg_speed'] = trips['trip_distance']/((trips['dpo_time']-trips['pku_time'])/timedelta(seconds=1)/3600)

In [15]:
# Only consider trips with average speed smaller than 100mph
trips = trips.loc[trips['avg_speed'] < 100.0, :]

In [16]:
# LGA coordinate
lga_long = -73.8702298524
lga_lat = 40.7730135746
lga_long_rad = lga_long*np.pi/180
lga_lat_rad = lga_lat*np.pi/180

In [17]:
# Function to calculate distance between pickup point and LGA position
def distanceToLGA_PKU(df):
    long_rad = df['pickup_longitude']*np.pi/180
    lat_rad = df['pickup_latitude']*np.pi/180
    drad = 2*np.arcsin(np.sqrt(np.sin((lat_rad-lga_lat_rad)/2)*np.sin((lat_rad-lga_lat_rad)/2)+np.cos(lat_rad)*np.cos(lga_lat_rad)*np.sin((long_rad-lga_long_rad)/2)*np.sin((long_rad-lga_long_rad)/2)))
    dmile = drad*180*60*1.15/np.pi
    return df.assign(pku_LGA=dmile)

In [18]:
# Function to calculate distance between dropoff point and LGA position
def distanceToLGA_DPO(df):
    long_rad = df['dropoff_longitude']*np.pi/180
    lat_rad = df['dropoff_latitude']*np.pi/180
    drad = 2*np.arcsin(np.sqrt(np.sin((lat_rad-lga_lat_rad)/2)*np.sin((lat_rad-lga_lat_rad)/2)+np.cos(lat_rad)*np.cos(lga_lat_rad)*np.sin((long_rad-lga_long_rad)/2)*np.sin((long_rad-lga_long_rad)/2)))
    dmile = drad*180*60*1.15/np.pi
    return df.assign(dpo_LGA=dmile)

In [19]:
lga_long = -73.8702298524
lga_lat = 40.7730135746

In [20]:
lga_long_rad = lga_long*np.pi/180
lga_lat_rad = lga_lat*np.pi/180

In [21]:
# New column: distance from pickup point to LGA
trips = trips.pipe(distanceToLGA_PKU)

In [22]:
# New column: distance from drop off point to LGA
trips = trips.pipe(distanceToLGA_DPO)

In [23]:
# Trips from LGA: pickup point within 1 mile range from LGA, and dropoff point outside 1 mile range of LGA
# Pickup time in range [May 17 2016 00:00, May 18 2016 00:00)
#trips_fromLGA = trips.loc[(trips['pku_LGA'] <= 1.0) & (trips['dpo_LGA'] > 1.0) & (trips['pku_time'] >= datetime(2016,5,17,0,0,0)) & (trips['pku_time'] < datetime(2016,5,18,0,0,0)), :]

In [24]:
# Create new graph
G = nx.Graph()

In [26]:
def build_graph_fromLGA(month, date):
    # Trips from LGA: pickup point within 1 mile range from LGA, and dropoff point outside 1 mile range of LGA
# Pickup time in range [May 17 2016 00:00, May 18 2016 00:00)
    trips_fromLGA = trips.loc[(trips['pku_LGA'] <= 1.0) & (trips['dpo_LGA'] > 1.0) & (trips['pku_time'] >= datetime(2016,month,date,0,0,0)) & (trips['pku_time'] < datetime(2016,month,date+1,0,0,0)), :]
    startTime = datetime(2016,month,date,0,0,0)
    # pool window: 5minute
    delta = timedelta(minutes=5)
    # array to store % miles saved for each pool
    milesaves = []
    # array to store % trips saved for each pool
    tripsaves = []
    # array to store average computing time for each pool
    computetimes = []
    # get time when start running
    start = datetime.now()
    for k in range(0, 288):
    #     get current time
        current = datetime.now()
    #     if pass 5 minutes, exit for loop
        if ((current-start)/timedelta(seconds=1)>300):
            break
    #   get pool for 5 minute window
        pool = trips_fromLGA.loc[(trips_fromLGA['pku_time'] >= startTime + k*delta) & (trips_fromLGA['pku_time'] < startTime + (k+1)*delta),:]
    #   if there are trips in pool
        if (len(pool) > 0):
    #       Clear graph
            G.clear()
    #       Add nodes to graph
            for n in range(len(pool)):
                G.add_node(n)
            saving = 0.0
            starttime = datetime.now()
            for i in range(len(pool) - 1):
                for j in range(i+1, len(pool)):
                    #               Calculate Euclidean distance between 2 dropoff points of 2 trips
#               Then multiplied by mean value of distance ratio
                    d = distanceTwoDPO(pool.iloc[i], pool.iloc[j])*1.6389485777
    #               Check if miles can be saved
                    if (d < pool.iloc[i].trip_distance) or (d < pool.iloc[j].trip_distance):
    #                   Calculate traveling time from Hub to A
                        tHA = (pool.iloc[i].dpo_time - pool.iloc[i].pku_time)/timedelta(seconds=1)
    #                   Calculate traveling time from Hub to B
                        tHB = (pool.iloc[j].dpo_time - pool.iloc[j].pku_time)/timedelta(seconds=1)
    #                   Calculate delay tolerance of trip HA
                        delayA = tHA * 0.2
    #                   Calculate delay tolerance of trip HB
                        delayB = tHB * 0.2
    #                   Calculate average speed of 2 trips
                        avgSpeed = (pool.iloc[i].avg_speed + pool.iloc[j].avg_speed)/2.0
    #                   Calculate time to travel from A to B, or from B to A
                        tAB = d * 3600 / avgSpeed
    #                   If 2 trips are sharable, add edge to graph
                        if (tHA + tAB <= tHB + delayB) or (tHB + tAB <= tHA + delayA):
                            saved = pool.iloc[i].trip_distance + pool.iloc[j].trip_distance - d
                            G.add_edge(i, j, weight=saved)
    #       Call the maximum matching on the graph
            matching = nx.algorithms.matching.max_weight_matching(G, maxcardinality=False, weight='weight')
            endtime = datetime.now()
    #       Calculate the total miles saved of each pool
            for m in matching:
                saving = saving + G.edges[m[0], m[1]]['weight']
    #       Add result to arrays
            milesaves.append(saving/pool['trip_distance'].sum())
            tripsaves.append(len(matching)/len(pool))
            computetimes.append((endtime-starttime)/timedelta(seconds=1))
    print("The number of pools get processed in 5 minutes: ", len(milesaves))
        # Average percent of trips saved per pool
    valid = []
    for trip in tripsaves:
        if trip > 0:
            valid.append(trip)
    print("Average percent of trips saved per pool: ", np.average(valid)*100)
    

In [27]:
def distanceTwoPKU(trip1, trip2):
    pku_long_rad = trip1.pickup_longitude*np.pi/180
    pku_lat_rad = trip1.pickup_latitude*np.pi/180
    dpo_long_rad = trip2.pickup_longitude*np.pi/180
    dpo_lat_rad = trip2.pickup_latitude*np.pi/180
    drad = 2*np.arcsin(np.sqrt(np.sin((pku_lat_rad-dpo_lat_rad)/2)*np.sin((pku_lat_rad-dpo_lat_rad)/2)+np.cos(pku_lat_rad)*np.cos(dpo_lat_rad)*np.sin((pku_long_rad-dpo_long_rad)/2)*np.sin((pku_long_rad-dpo_long_rad)/2)))
    dmile = drad*180*60*1.15/np.pi
    return dmile

In [28]:
#trips_toLGA = trips.loc[(trips['pku_LGA'] > 1.0) & (trips['dpo_LGA'] <= 1.0) & (trips['dpo_time'] >= datetime(2016,5,17,0,0,0)) & (trips['dpo_time'] < datetime(2016,5,18,0,0,0)), :]

In [44]:
def build_graph_toLGA(month, date):
    trips_toLGA = trips.loc[(trips['pku_LGA'] > 1.0) & (trips['dpo_LGA'] <= 1.0) & (trips['dpo_time'] >= datetime(2016,month,date,0,0,0)) & (trips['dpo_time'] < datetime(2016,month,date+1,0,0,0)), :]
    startTime = datetime(2016,month,date,0,0,0)
    delta = timedelta(minutes=5)
    milesaves_LGA = []
    tripsaves_LGA = []
    computetimes_LGA = []
    # get time when start running
    start = datetime.now()
    for k in range(0, 288):
    #   get current time
        current = datetime.now()
    #   if pass 5 minutes, exit for loop
        if ((current-start)/timedelta(seconds=1)>300):
            break
        pool = trips_toLGA.loc[(trips_toLGA['dpo_time'] >= startTime + k*delta) & (trips_toLGA['dpo_time'] < startTime + (k+1)*delta),:]
        if (len(pool) > 0):
            G.clear()
            for n in range(len(pool)):
                G.add_node(n)
            saving = 0.0
            starttime = datetime.now()
            for i in range(len(pool) - 1):
                for j in range(i+1, len(pool)):
                    d = distanceTwoPKU(pool.iloc[i], pool.iloc[j])*1.679833796569822
                    if (d < pool.iloc[i].trip_distance) or (d < pool.iloc[j].trip_distance):
                        tHA = (pool.iloc[i].dpo_time - pool.iloc[i].pku_time)/timedelta(seconds=1)
                        tHB = (pool.iloc[j].dpo_time - pool.iloc[j].pku_time)/timedelta(seconds=1)
                        delayA = tHA * 0.2
                        delayB = tHB * 0.2
                        avgSpeed = (pool.iloc[i].avg_speed + pool.iloc[j].avg_speed)/2.0
                        tAB = d * 3600 / avgSpeed
                        if (pool.iloc[i].pku_time <= pool.iloc[j].pku_time):
                            delta_PKU = (pool.iloc[j].pku_time - pool.iloc[i].pku_time)/timedelta(seconds=1)
                            if (tAB <= delta_PKU + delayB) and (tAB + tHB <= tHA + delayA):
                                saved = pool.iloc[i].trip_distance + pool.iloc[j].trip_distance - d
                                G.add_edge(i, j, weight=saved)
                        else:
                            delta_PKU = (pool.iloc[i].pku_time - pool.iloc[j].pku_time)/timedelta(seconds=1)
                            if (tAB <= delta_PKU + delayA) and (tAB + tHA <= tHB + delayB):
                                saved = pool.iloc[i].trip_distance + pool.iloc[j].trip_distance - d
                                G.add_edge(i, j, weight=saved)
            matching = nx.algorithms.matching.max_weight_matching(G, maxcardinality=False, weight='weight')
            endtime = datetime.now()
            for m in matching:
                saving = saving + G.edges[m[0], m[1]]['weight']
            milesaves_LGA.append(saving/pool['trip_distance'].sum())
            tripsaves_LGA.append(len(matching)/len(pool))
            computetimes_LGA.append((endtime-starttime)/timedelta(seconds=1))
    print("The number of pools get processed in 5 minutes: ", len(milesaves_LGA))
    # Average percent of trips saved per pool
    valid = []
    for trip in tripsaves_LGA:
        if trip > 0:
            valid.append(trip)
    print("Average percent of trips saved per pool: ", np.average(valid)*100)

In [37]:
# Run fromLGA algorithm, paramaters are month, date
build_graph_fromLGA(5, 17)

The number of pools get processed in 5 minutes:  90
Average percent of trips saved per pool:  46.954324510633796
--- 306.74683713912964 seconds ---


In [45]:
# Run toLGA algorithm, paramaters are month, date
build_graph_toLGA(5, 17)

The number of pools get processed in 5 minutes:  146
Average percent of trips saved per pool:  47.50832354501809
